# Compressing Data via Dimensionality Reduction

In this notebook, we will cover the following topics:-

- **Principal Component Analysis (PCA)** for unsupervised data compression
- **Linear Discriminant Analysis (LDA)** for supervised dimensionality reduction technique for maximizing class separability
- Nonlinear dimensionality reduction vis **Kernel Principal Component Analysis (KPCA)**

# PCA

PCA is an unsupervised *linear transformation* technique that is widely used across different fields, most prominently for feature extraction and dimensionality reduction. Other popular areas of PCA are EDA (we will only talk about area of PCA in ml, but it is also used in de-noising of signals, genome data analysis, etc)

PCA helps us to identify patterns in data based on the *correlation* between features.

PCA aims to find the directions of maximum variance in high dimensional data and projects it onto a new subspace with equal or fewer dimensions than the original one.

*Steps to perform PCA*

1. Standarize the data.
2. Constructing the covariance matrix.
3. Obtaining the eigenvalues and eigenvectors of the covariance matrix.
4. Sorting the eigenvalues by decreasing order to rank the eigen vectors.

**Covariance Matrix**



In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('wine.data', header = None)

In [3]:
#splitting the dataset, in ratio of 70:30 and also making sure that proportions of classes are equal in train and test set
from sklearn.model_selection import train_test_split